In [ ]:
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [8]:
tif_path = "../../data/HydroSHEDS/as_acc_3s.tif"

In [9]:
with rasterio.open(tif_path) as src:
    print("📌 CRS (Coordinate Reference System):", src.crs)
    print("📐 Size (width x height):", src.width, "x", src.height)
    print("📦 Number of bands:", src.count)
    print("🧭 Affine transformation (pixel → geographic coordinates):\n", src.transform)

📌 CRS (Coordinate Reference System): EPSG:4326
📐 Size (width x height): 156000 x 72000
📦 Number of bands: 1
🧭 Affine transformation (pixel → geographic coordinates):
 | 0.00, 0.00, 50.00|
| 0.00,-0.00, 60.00|
| 0.00, 0.00, 1.00|


In [ ]:
with rasterio.open(tif_path) as src:
    scale = 20
    new_height = src.height // scale
    new_width = src.width // scale
    data = src.read(
        1,
        out_shape=(1, new_height, new_width),
        resampling=rasterio.enums.Resampling.average
    )
    data = np.where(data <= 0, np.nan, data)

plt.figure(figsize=(10, 6))
plt.imshow(data, cmap='viridis')
plt.title("Downsampled Flow Accumulation")
plt.colorbar(label="Flow Accumulation")
plt.show()


NameError: name 'window' is not defined

In [ ]:
with rasterio.open(tif_path) as src:
    band = src.read(1)
    transform = src.transform

# select only valid (non-zero) values
rows, cols = np.where(band > 0)
values = band[rows, cols]

# convert pixel coordinates to geographic coordinates
xs, ys = rasterio.transform.xy(transform, rows, cols)

# create a DataFrame
df = pd.DataFrame({
    "longitude": xs,
    "latitude": ys,
    "flow_accumulation": values
})

df.head()


In [ ]:
csv_path = "../../data/HydroSHEDS/as_acc_3s.csv"
df.to_csv(csv_path, index=False)
print(f"✅ CSV saved to {csv_path}")